In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [3]:
pwd

'C:\\Users\\vinot\\Fintech\\LSTM\\Bitcoin_Price_Prediction'

In [4]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df.head()

,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42


In [9]:
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [10]:

# Load the historical closing prices for bitcoin
df2 = pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2.tail()

Date
2019-07-25    9882.429688
2019-07-26    9847.450195
2019-07-27    9478.320313
2019-07-28    9531.769531
2019-07-29    9529.889648
Name: Close, dtype: float64

In [11]:
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [12]:
window_size = 1
feature_column = 1
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

In [13]:
split = int(0.7 * len(X))
X_train = X[: split - 1]
X_test = X[split:]
y_train = y[: split - 1]
y_test = y[split:]

In [14]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
scaler.fit(y)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)
X_train[:5]

array([[0.60761794],
       [0.58242373],
       [0.62172321],
       [0.51499412],
       [0.38267307]])

In [15]:
# Reshape the features for the model
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
print (f"X_train sample values:\n{X_train[:5]} \n")
print (f"X_test sample values:\n{X_test[:5]}")

X_train sample values:
[[[0.60761794]]

 [[0.58242373]]

 [[0.62172321]]

 [[0.51499412]]

 [[0.38267307]]] 

X_test sample values:
[[[0.03974167]]

 [[0.04528668]]

 [[0.04528668]]

 [[0.07024855]]

 [[0.07145402]]]


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [17]:
model = Sequential()

number_units = 30
dropout_fraction = 0.2

# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))
# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))
# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))
# Output layer
model.add(Dense(1))

In [18]:
model.compile(optimizer="adam", loss="mean_squared_error")

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 30)             3840      
_________________________________________________________________
dropout (Dropout)            (None, 1, 30)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 30)             7320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 30)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 30)                7320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 3

In [26]:
model.fit(X_train, y_train, epochs=50, shuffle=False, batch_size=1, verbose=1)

Epoch 1/50
377/377 [==============================] - 1s 2ms/step - loss: 0.0030
Epoch 2/50
377/377 [==============================] - 1s 2ms/step - loss: 0.0044
Epoch 3/50
377/377 [==============================] - 1s 2ms/step - loss: 0.0043
Epoch 4/50
377/377 [==============================] - 1s 2ms/step - loss: 0.0043
Epoch 5/50
377/377 [==============================] - 1s 2ms/step - loss: 0.0042
Epoch 6/50
377/377 [==============================] - 1s 2ms/step - loss: 0.0043
Epoch 7/50
377/377 [==============================] - 1s 2ms/step - loss: 0.0038
Epoch 8/50
377/377 [==============================] - 1s 2ms/step - loss: 0.0046
Epoch 9/50
377/377 [==============================] - 1s 2ms/step - loss: 0.0039
Epoch 10/50
377/377 [==============================] - 1s 2ms/step - loss: 0.0044
Epoch 11/50
377/377 [==============================] - 1s 2ms/step - loss: 0.0040
Epoch 12/50
377/377 [==============================] - 1s 2ms/step - loss: 0.0040
Epoch 13/50
377/377 [====

In [29]:
model.evaluate(X_test, y_test)

6/6 [==============================] - 0s 2ms/step - loss: 0.0144


0.014424326829612255

In [31]:
predicted = model.predict(X_test)

In [32]:
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

In [33]:
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
})
stocks.head()

,Real,Predicted
0,3670.919922,3589.558350
1,3670.919922,3619.332764
2,3912.570068,3619.332764
3,3924.239990,3758.006104
4,3974.050049,3764.896973


In [ ]:
stocks.plot(title="Real vs. Predicted Stock Price", xlabel="Days", ylabel="Price")

<AxesSubplot:title={'center':'Real vs. Predicted Stock Price'}, xlabel='Days', ylabel='Price'>